# Breakfast at the Frat: A Time Series Analysis

Sales and promotion information on the top five products from each of the top three brands within four selected categories (mouthwash, pretzels, frozen pizza, and boxed cereal), gathered from a sample of stores over 156 weeks.

- Unit sales, households, visits, and spend data by product, store, and week
- Base Price and Actual Shelf Price, to determine a product’s discount, if any
- Promotional support details (e.g., sale tag, in-store display), if applicable for the given product/store/week
- Store information, including size and location, as well as a price tier designation (e.g., upscale vs. value)
- Product information, including UPC, size, and description

To identify outliers, it is suggested to look at

- The ratio of units vs. number of visits
- The ratio of visits vs. number of households
- Some items that may be out-of-stock or discontinued for a store

**Source:** https://www.dunnhumby.com/source-files/

In [29]:
from pyspark.sql import SparkSession

In [30]:
spark = SparkSession.builder \
    .appName("breakfast") \
    .getOrCreate()

In [31]:
product_data_folder = "dataset/products"
store_data_folder = "dataset/stores"
transaction_data_folder = "dataset/transactions"

In [32]:
#Sandbox Session

### Perform ETL to Answer the Following Questions

1. What is the range of prices offered on products?
1. What is the impact on units/visit of promotions by geographies?
1. Which products would you lower the price to increase sales?

In [33]:
product_df = spark. \
    read. \
    option("header",True) \
    .csv(product_data_folder)

In [34]:
product_df.show(1)

+----------+--------------------+-------------+----------+------------+------------+
|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|
+----------+--------------------+-------------+----------+------------+------------+
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
+----------+--------------------+-------------+----------+------------+------------+
only showing top 1 row



In [35]:
product_df.show()

+----------+--------------------+-------------+--------------------+--------------------+------------+
|       UPC|         DESCRIPTION| MANUFACTURER|            CATEGORY|        SUB_CATEGORY|PRODUCT_SIZE|
+----------+--------------------+-------------+--------------------+--------------------+------------+
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|          BAG SNACKS|            PRETZELS|       15 OZ|
|1111009497|   PL PRETZEL STICKS|PRIVATE LABEL|          BAG SNACKS|            PRETZELS|       15 OZ|
|1111009507|   PL TWIST PRETZELS|PRIVATE LABEL|          BAG SNACKS|            PRETZELS|       15 OZ|
|1111035398|PL BL MINT ANTSPT...|PRIVATE LABEL|ORAL HYGIENE PROD...|MOUTHWASHES (ANTI...|      1.5 LT|
|1111038078|PL BL MINT ANTSPT...|PRIVATE LABEL|ORAL HYGIENE PROD...|MOUTHWASHES (ANTI...|      500 ML|
|1111038080|PL ANTSPTC SPG MN...|PRIVATE LABEL|ORAL HYGIENE PROD...|MOUTHWASHES (ANTI...|      500 ML|
|1111085319|PL HONEY NUT TOAS...|PRIVATE LABEL|         COLD CEREAL|   AL

In [36]:
store_df = spark. \
    read. \
    option("header",True) \
    .csv(store_data_folder)

In [37]:
product_df.show(3)

+----------+--------------------+-------------+----------+------------+------------+
|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|
+----------+--------------------+-------------+----------+------------+------------+
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
|1111009497|   PL PRETZEL STICKS|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
|1111009507|   PL TWIST PRETZELS|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
+----------+--------------------+-------------+----------+------------+------------+
only showing top 3 rows



In [38]:
transaction_df = spark. \
    read. \
    option("header",True) \
    .csv(transaction_data_folder)

In [39]:
transaction_df.show(3)

+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+
|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS|SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+
|    14-Jan-09|      367|1111009477|   13|    13| 13|18.07| 1.39|      1.57|      0|      0|       1|
|    14-Jan-09|      367|1111009497|   20|    18| 18| 27.8| 1.39|      1.39|      0|      0|       0|
|    14-Jan-09|      367|1111009507|   14|    14| 14|19.32| 1.38|      1.38|      0|      0|       0|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+
only showing top 3 rows



In [40]:
product_df.createOrReplaceTempView("products")
transaction_df.createOrReplaceTempView("transaction")

In [41]:
spark.sql("""
    SELECT
        *
        
    FROM transaction
    JOIN products
    ON
        transaction.upc = products.upc
""").show(3)

+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS|SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
|    14-Jan-09|      367|1111009477|   13|    13| 13|18.07| 1.39|      1.57|      0|      0|       1|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
|    14-Jan-09|      367|1111009497|   20|    18| 18| 27.8| 1.39|      1.39|      0|      0|       0|1111009497|   PL PRETZEL STICKS|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
|    14-Jan-09|      367|1111009507|   14|    14| 14|19.32| 1.38|      1.38

In [42]:
df = spark.sql("""
    SELECT
        *
        
    FROM transaction
    JOIN products
    ON
        transaction.upc = products.upc
""")

In [43]:
df.show(3)

+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS|SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
|    14-Jan-09|      367|1111009477|   13|    13| 13|18.07| 1.39|      1.57|      0|      0|       1|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
|    14-Jan-09|      367|1111009497|   20|    18| 18| 27.8| 1.39|      1.39|      0|      0|       0|1111009497|   PL PRETZEL STICKS|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
|    14-Jan-09|      367|1111009507|   14|    14| 14|19.32| 1.38|      1.38

In [44]:
spark.sql("""
    SELECT
        products.upc
        , price
        , description
        , category
        
    FROM transaction
    JOIN products
    ON
        transaction.upc = products.upc
""").show(3)

+----------+-----+--------------------+----------+
|       upc|price|         description|  category|
+----------+-----+--------------------+----------+
|1111009477| 1.39|PL MINI TWIST PRE...|BAG SNACKS|
|1111009497| 1.39|   PL PRETZEL STICKS|BAG SNACKS|
|1111009507| 1.38|   PL TWIST PRETZELS|BAG SNACKS|
+----------+-----+--------------------+----------+
only showing top 3 rows



In [45]:
spark.sql("""
    SELECT
        products.upc
        , min (price)
        , max (price)
        , description
        , category
        
    FROM transaction
    JOIN products
    ON
        transaction.upc = products.upc
    GROUP BY
        1, 4, 5
""").show(3)

+----------+----------+----------+--------------------+----------+
|       upc|min(price)|max(price)|         description|  category|
+----------+----------+----------+--------------------+----------+
|1111009477|      0.89|      1.83|PL MINI TWIST PRE...|BAG SNACKS|
|1111009497|      0.86|      1.69|   PL PRETZEL STICKS|BAG SNACKS|
|1111009507|       0.8|      1.69|   PL TWIST PRETZELS|BAG SNACKS|
+----------+----------+----------+--------------------+----------+
only showing top 3 rows



In [46]:
spark.sql("""
    SELECT
        upc
        , product_size
        , CASE
            WHEN CONTAINS(product_size, 'OZ') THEN 'yes'
        END AS is_oz
        
    FROM products
""").show(10)

+----------+------------+-----+
|       upc|product_size|is_oz|
+----------+------------+-----+
|1111009477|       15 OZ|  yes|
|1111009497|       15 OZ|  yes|
|1111009507|       15 OZ|  yes|
|1111035398|      1.5 LT| null|
|1111038078|      500 ML| null|
|1111038080|      500 ML| null|
|1111085319|    12.25 OZ|  yes|
|1111085345|       20 OZ|  yes|
|1111085350|       18 OZ|  yes|
|1111087395|     32.7 OZ|  yes|
+----------+------------+-----+
only showing top 10 rows



In [47]:
product_df.createOrReplaceTempView("products")
store_df.createOrReplaceTempView("stores")
transaction_df.createOrReplaceTempView("transactions")

In [48]:
#--------------------------------------------------------------------

In [49]:
#HW Session

In [50]:
#ช่วงราคาสินค้าแต่ะละประเภท

In [51]:
spark.sql("""
    select
        products.upc
        , min(price)
        , max(price)
        , description
        , category
        
    from transactions
    join products
    on 
        transactions.upc = products.upc
    group by
        1, 4, 5
""").show(30)

+-----------+----------+----------+--------------------+--------------------+
|        upc|min(price)|max(price)|         description|            category|
+-----------+----------+----------+--------------------+--------------------+
| 1111009477|      0.89|      1.83|PL MINI TWIST PRE...|          BAG SNACKS|
| 1111009497|      0.86|      1.69|   PL PRETZEL STICKS|          BAG SNACKS|
| 1111009507|       0.8|      1.69|   PL TWIST PRETZELS|          BAG SNACKS|
| 1111035398|         1|      4.69|PL BL MINT ANTSPT...|ORAL HYGIENE PROD...|
| 1111038078|      0.47|      3.08|PL BL MINT ANTSPT...|ORAL HYGIENE PROD...|
| 1111038080|      0.46|      4.18|PL ANTSPTC SPG MN...|ORAL HYGIENE PROD...|
| 1111085319|      1.07|      1.99|PL HONEY NUT TOAS...|         COLD CEREAL|
| 1111085345|      0.93|         2|      PL RAISIN BRAN|         COLD CEREAL|
| 1111085350|      1.03|      2.49|PL BT SZ FRSTD SH...|         COLD CEREAL|
| 1111087395|       1.4|      5.29|PL SR CRUST SUPRM...|        

In [52]:
spark.sql("""
    select
        upc
        , product_size
        , split(product_size, ' ')[0] as value
        , split(product_size, ' ')[1] as unit
        , case
            when split(product_size, ' ')[1] = 'OZ' then split(product_size, ' ')[0] * 100
            else
                split(product_size, ' ')[0] / 100
            end as is_oz
        
    from products
""").show(50)

+----------+------------+-----+----+------------------+
|       upc|product_size|value|unit|             is_oz|
+----------+------------+-----+----+------------------+
|1111009477|       15 OZ|   15|  OZ|            1500.0|
|1111009497|       15 OZ|   15|  OZ|            1500.0|
|1111009507|       15 OZ|   15|  OZ|            1500.0|
|1111035398|      1.5 LT|  1.5|  LT|             0.015|
|1111038078|      500 ML|  500|  ML|               5.0|
|1111038080|      500 ML|  500|  ML|               5.0|
|1111085319|    12.25 OZ|12.25|  OZ|            1225.0|
|1111085345|       20 OZ|   20|  OZ|            2000.0|
|1111085350|       18 OZ|   18|  OZ|            1800.0|
|1111087395|     32.7 OZ| 32.7|  OZ|3270.0000000000005|
|1111087396|     30.5 OZ| 30.5|  OZ|            3050.0|
|1111087398|     29.6 OZ| 29.6|  OZ|            2960.0|
|1600027527|    12.25 OZ|12.25|  OZ|            1225.0|
|1600027528|       18 OZ|   18|  OZ|            1800.0|
|1600027564|       12 OZ|   12|  OZ|            

In [53]:
#โปรโมชั่นที่ส่งผลต่อยอดขายตามพื้นที่ภูมิศาสตร์

In [54]:
spark.sql("""
    select
        -- transactions.upc
        -- , transactions.STORE_NUM
        -- , price
        -- , base_price
        case 
            when (base_price - price) > 0  then 1 
            else 
                0
            end as is_sale
        -- , units
        -- , visits
        , units/visits as buy_per_p
        
        , ADDRESS_STATE_PROV_CODE
        
        
    from transactions
    join stores
    on 
        transactions.STORE_NUM = stores.STORE_ID
    where (base_price - price) > 0

    
""").show(50)

+-------+------------------+-----------------------+
|is_sale|         buy_per_p|ADDRESS_STATE_PROV_CODE|
+-------+------------------+-----------------------+
|      1|               1.0|                     KY|
|      1|1.3333333333333333|                     KY|
|      1|              1.25|                     KY|
|      1|1.4186046511627908|                     KY|
|      1|1.3846153846153846|                     KY|
|      1|1.1666666666666667|                     KY|
|      1|1.1176470588235294|                     KY|
|      1| 1.144736842105263|                     KY|
|      1| 1.170731707317073|                     KY|
|      1|               1.0|                     KY|
|      1|1.0172413793103448|                     KY|
|      1|               1.0|                     KY|
|      1|             1.625|                     KY|
|      1|             1.375|                     KY|
|      1|1.3333333333333333|                     KY|
|      1|1.0454545454545454|                  

In [55]:
spark.sql("""
    select        
           
        AVG(units/visits) as buy_per_p        
        , ADDRESS_STATE_PROV_CODE        
        
    from transactions
    join stores
    on 
        transactions.STORE_NUM = stores.STORE_ID
    where (base_price - price) > 0
    group by
        ADDRESS_STATE_PROV_CODE

    
""").show(50)

+------------------+-----------------------+
|         buy_per_p|ADDRESS_STATE_PROV_CODE|
+------------------+-----------------------+
|1.1834336090000268|                     KY|
|1.1928337487593017|                     IN|
|1.1898599127461704|                     OH|
|1.1523177900229675|                     TX|
+------------------+-----------------------+



In [56]:
#สินค้าที่กระตุ้นยอดขายจากการลดราคา

In [57]:
spark.sql("""
    select
       
        CATEGORY
        -- , sub_CATEGORY
        , AVG(units/visits) as buy_per_p
        -- , SPEND
        
    from transactions
    join products
    on 
        transactions.upc = products.upc
    where (base_price - price) > 0
    group by
        CATEGORY

    
""").show(50)

+--------------------+------------------+
|            CATEGORY|         buy_per_p|
+--------------------+------------------+
|         COLD CEREAL|1.1681271661328343|
|        FROZEN PIZZA|1.1645997816954927|
|ORAL HYGIENE PROD...| 1.121139647543165|
|          BAG SNACKS|1.2368985083204924|
+--------------------+------------------+

